In [ ]:
#Scraping of data from the Open Source Library of the National Gallery of Denmark (SMK). Check out: https://www.smk.dk/en/article/smk-open/
import requests
import torch
from PIL import Image
import io 
from IPython.display import clear_output
import json

In [ ]:
#If you're working with Colab mount your drive or skip this step
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
respone = requests.get('https://api.smk.dk/api/v1/art/all_ids')

In [ ]:
import json
ids = json.loads(respone.content)['objectIDs']

In [ ]:
print(len(ids))

In [ ]:
import os
path = "/create/a/new/path/for/the/scraped/images/"
if not os.path.exists(path):
    os.makedirs(path)

In [ ]:
rights_file = '/set/a/path/for/the/file/is_public.json'
processed_ids_file = '/set/a/path/for/the/file/processed_ids.json'

In [ ]:
links = {}
if os.path.exists(processed_ids_file):
    is_public = json.load(open(rights_file))
    processed_ids = json.load(open(processed_ids_file))
else:
    processed_ids = []
    is_public = {}

download_image = False
chunk = set(ids).difference(processed_ids)

to_process = len(chunk)
links_found = 0
for ix, oid in enumerate(chunk):  
    manifest = requests.get(f'https://api.smk.dk/api/v1/art/?object_number={oid}')
    manifest = json.loads(manifest.content)
    for m in manifest['items']:
        l = m.get('image_thumbnail', False)
        if l:
            try:
                onr = m['object_number']
                image_name = f"{onr.replace('/','_')}.jpg"
                links_found += 1
                if download_image:
                    image = requests.get(l)
                    image = Image.open(io.BytesIO(image.content))
                    image.save(os.path.join(path, image_name))
                is_public[image_name] = m.get('public_domain', None) #this helps to filter images with a copyright later on
            except:
                print('failed to read image')
    processed_ids.append(oid)
    if ix%100 == 0:
        json.dump(is_public, open(rights_file, 'w'))
        json.dump(processed_ids, open(processed_ids_file, 'w'))
        clear_output(wait=True)
        print(f'{to_process} ids need processing, {links_found} links extracted')
    to_process -= 1